# <h1 align="center"><font color="gree">Mistral AI Agents API Tutorial</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [Obtenha a sua chave de API do Mistral AI](https://console.mistral.ai/api-keys)

* [DataCamp - Mistral AI](https://www.youtube.com/watch?v=VkqYmwIIeOg)

* [Mistral AI Cookbook](https://github.com/mistralai/cookbook/tree/main)

* [Visão geral de modelos](https://docs.mistral.ai/getting-started/models/models_overview/)

# <font color="red">Começando</font>

In [1]:
from mistralai import Mistral
import os

# Alternativamente, você pode usar variáveis de ambiente
MISTRALAI_API_KEY = os.environ.get("MISTRALAI_API_KEY") 

client = Mistral(api_key=MISTRALAI_API_KEY)

ml_agent = client.beta.agents.create(
    model="mistral-medium-2505",
    name="ml-agent",
    description="Asistente de Machine learning",
    instructions="Você é um especialista em Machine learning. Dá conselhos práticos e ações.",
)


In [5]:
print(f"Agent created: {ml_agent.id}")
print("\n")
print(f"Agent name: {ml_agent.name}")
print("\n")
print(f"Agent description: {ml_agent.description}")
print("\n")
print(f"Agent instructions: {ml_agent.instructions}")
print("\n")
print(f"Agent model: {ml_agent.model}")


Agent created: ag_019818e32a1c73d88638d1a73a709ff8


Agent name: ml-agent


Agent description: Asistente de Machine learning


Agent instructions: Você é um especialista em Machine learning. Dá conselhos práticos e ações.


Agent model: mistral-medium-2505


* ``mistral-medium`` é um modelo pago com um equilíbrio entre desempenho e custo
* ``name`` deve ser único para manter conversas separadas
* ``instructions`` é o prompt do sistema que guia o comportamento do agente

Os Agents API usam IDs de agentes para diferenciar um agente de outro


<font color="gree">Iniciando uma conversa</font>

* Para começar a fazer perguntas, use o comando ``start``
* Deve passar o ``ID`` de agente


In [6]:
response = client.beta.conversations.start(
    agent_id=ml_agent.id,
    inputs="""Devo usar Random Forest ou XGBoost para um dataset de 
              5000 amostras? Responda em uma sentença apenas.""",
)

In [7]:
type(response)

mistralai.models.conversationresponse.ConversationResponse

In [ ]:
response.outputs

[MessageOutputEntry(content='Para um dataset de 5000 amostras, XGBoost geralmente oferece melhor desempenho e eficiência.', object='entry', type='message.output', created_at=datetime.datetime(2025, 7, 17, 15, 6, 14, 383662, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2025, 7, 17, 15, 6, 14, 708149, tzinfo=TzInfo(UTC)), id='msg_019818ebf7ef72b79a5092bf66a696eb', agent_id='ag_019818e32a1c73d88638d1a73a709ff8', model='mistral-medium-2505', role='assistant')]

* O array de saídas (``outputs``) conterá outros objetos de mensagem como chamadas de ferramentas.

In [11]:
print(response.outputs[0].content)

Para um dataset de 5000 amostras, XGBoost geralmente oferece melhor desempenho e eficiência.


Isto é um workflow básico para criar agentes e iniciar conversas com eles.


# <font color="red">Mergulho profundo na API de agentes Mistral</font>

## <font color="gree">``1.`` Criando agentes eficazes</font>

* O desempenho do agente é determinado por:

1. Escolha do modelo
2. Prompt do sistema
3. Parâmetros de conclusão (`Completion`)

* Escolha o modelo certo na página de [visão geral de modelos](https://docs.mistral.ai/getting-started/models/models_overview/)


In [ ]:
# Agentes especializados com diferentes configurações:
data_agent = client.beta.agents.create(
    model="mistral-medium-latest",
    name="data-expert",
    description="Especialista em pré-processamento de dados",
    instructions="Especialista em limpeza de dados e engenharia de recursos (feature engineering)",
    completion_args={"temperature": 0.1},
)

print(f"Data agent: {data_agent.id}")

Data agent: ag_01981910206d75709f9cf966b4e184f3


In [13]:
print(data_agent)

model='mistral-medium-latest' name='data-expert' id='ag_01981910206d75709f9cf966b4e184f3' version=0 created_at=datetime.datetime(2025, 7, 17, 15, 45, 44, 49338, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2025, 7, 17, 15, 45, 44, 49340, tzinfo=TzInfo(UTC)) instructions='Especialista em limpeza de dados e engenharia de recursos (feature engineering)' tools=[] completion_args=CompletionArgs(stop=None, presence_penalty=None, frequency_penalty=None, temperature=0.0, top_p=None, max_tokens=None, random_seed=None, prediction=None, response_format=None, tool_choice='auto') description='Especialista em pré-processamento de dados' handoffs=None object='agent'


* Use modelos premium (pagos) para cenários de alto impacto que exigem forte ``raciocínio/codificação``. Boas opções são:
    * ``Codestral 2`` para codificação
    * ``Magistral medium`` para raciocínio
* Faça o ``prompt do sistema`` o mais detalhado possível - dedique a maior parte do seu tempo aqui
* A ``temperatura`` controla a aleatoriedade da próxima palavra
    * ``0.1-0.5`` para precisão factual (codificação, documentos técnicos)
    * ``0.5-1`` para criatividade (brainstorming (gerar ideias), menos robótico)

## <font color="gree">``2.``Gerenciando conversas com um agente</font>

* A maioria dos frameworks de construção de agentes não possui memória embutida
* Os casos de uso em produção são diferentes
* O ``Mistral`` vem com memória embutida e gerenciamento de conversas

<font color="red">``I.`` Iniciando uma nova conversa</font>

* O comando ``start`` sempre espera um novo ID de agente

In [14]:
response = client.beta.conversations.start(
    agent_id=ml_agent.id,
    inputs="""Quais são as melhores práticas para codificar variáveis 
              categóricas em aprendizado de máquina? Liste três.""",
)

print(response.outputs[0].content)

Aqui estão três melhores práticas para codificar variáveis categóricas em aprendizado de máquina:

1. **One-Hot Encoding**:
   - **Descrição**: Transforma cada categoria em uma coluna binária (0 ou 1). Cada coluna representa uma categoria única.
   - **Quando usar**: Quando as categorias não têm uma ordem intrínseca (por exemplo, cores, países).
   - **Exemplo**: Se você tem uma variável "Cor" com categorias "Vermelho", "Azul" e "Verde", o One-Hot Encoding criará três colunas binárias, cada uma representando uma dessas cores.

2. **Label Encoding**:
   - **Descrição**: Atribui um número inteiro único a cada categoria.
   - **Quando usar**: Quando as categorias têm uma ordem intrínseca (por exemplo, níveis de educação: "Ensino Fundamental", "Ensino Médio", "Ensino Superior").
   - **Exemplo**: Se você tem uma variável "Nível de Educação" com categorias "Ensino Fundamental", "Ensino Médio" e "Ensino Superior", o Label Encoding pode atribuir 0, 1 e 2, respectivamente.

3. **Embedding**:
 

* Todos os objetos de ``response`` de conversação também têm um ``ID``.

In [15]:
print(response.conversation_id)

conv_01981929efd671908fe477e0bc9cf15c


<font color="red">``II.`` Continuando uma conversa existente</font>

In [16]:
# Continuar uma conversa existente:
follow_up = client.beta.conversations.append(
    conversation_id=response.conversation_id,
    inputs="Qual foi a minha primeira pergunta?",
)

print(follow_up.outputs[0].content)

Sua primeira pergunta foi: "Quais são as melhores práticas para codificar variáveis categóricas em aprendizado de máquina? Liste três."


In [17]:
follow_up = client.beta.conversations.append(
    conversation_id=response.conversation_id,
    inputs="Resuma toda a conversa em 3 frases.",
)

print(follow_up.outputs[0].content)

Você perguntou sobre as melhores práticas para codificar variáveis categóricas em aprendizado de máquina. Eu listei três técnicas: One-Hot Encoding, Label Encoding e Embedding. Em seguida, você perguntou qual foi a sua primeira pergunta.


<font color="red">``III.`` Listando todas as conversas</font>

* Para gerenciamento interno, você pode ver todas as conversas dos agentes rapidamente.

In [18]:
conversations_list = client.beta.conversations.list()

In [19]:
len(conversations_list)

3

In [20]:
conversations_list[0].id

'conv_01981929efd671908fe477e0bc9cf15c'

* As conversas mais recentes vêm primeiro nesta lista.
* Você pode obter o histórico completo de mensagens com ``get_messages``

In [21]:
conversation = client.beta.conversations.get_messages(conversation_id=conversations_list[0].id)

conversation.messages

[MessageInputEntry(role='user', content='Quais são as melhores práticas para codificar variáveis \n              categóricas em aprendizado de máquina? Liste três.', object='entry', type='message.input', created_at=datetime.datetime(2025, 7, 17, 16, 13, 55, 541356, tzinfo=TzInfo(UTC)), completed_at=None, id='msg_01981929efd5774395e0ea813cc7f090', prefix=False),
 MessageOutputEntry(content='Aqui estão três melhores práticas para codificar variáveis categóricas em aprendizado de máquina:\n\n1. **One-Hot Encoding**:\n   - **Descrição**: Transforma cada categoria em uma coluna binária (0 ou 1). Cada coluna representa uma categoria única.\n   - **Quando usar**: Quando as categorias não têm uma ordem intrínseca (por exemplo, cores, países).\n   - **Exemplo**: Se você tem uma variável "Cor" com categorias "Vermelho", "Azul" e "Verde", o One-Hot Encoding criará três colunas binárias, cada uma representando uma dessas cores.\n\n2. **Label Encoding**:\n   - **Descrição**: Atribui um número intei

In [22]:
len(conversation.messages)

6

* Importante ao construir aplicativos de chat (bate-papo) com histórico de mensagens
* Veja outros [métodos de gerenciamento de conversa](https://docs.mistral.ai/agents/agents_basics/#conversations) na documentação

## <font color="gree">``3.`` Respostas de streaming para facilidade de uso</font>

* Os agentes devem ser amigáveis.
* Transmita respostas sem fazê-los esperar.

In [23]:
message = "Explique a descida do gradiente em termos simples em 2 frases."

response = client.beta.conversations.start_stream(
    agent_id=ml_agent.id, inputs=message
)

In [24]:
type(response)

mistralai.utils.eventstreaming.EventStream

In [25]:
print(response)

In [26]:
# Usando o stream de eventos:
with response as event_stream:
    for event in event_stream:
        # Verifique o tipo de evento:
        if event.event == "message.output.delta":
            print(event.data.content, flush=True, end="")
            

A descida do gradiente é como descer uma montanha aos poucos, sempre escolhendo o caminho mais íngreme para baixo. Em machine learning, usamos essa técnica para ajustar os parâmetros do modelo de forma a minimizar o erro, ou seja, encontrar o melhor "vale" que representa a solução ótima.

* ``start_stream`` cria uma nova conversa como ``conversations.start``
* ``append_stream`` continua essa conversa

In [27]:
# Obter o ID da última conversa para continuar:
last_conversation_id = client.beta.conversations.list()[0].id

message = "Traduza sua última resposta para o Espanhol."
follow_up = client.beta.conversations.append_stream(
    conversation_id=last_conversation_id, inputs=message
)

with follow_up as event_stream:
    for event in event_stream:
        if event.event == "message.output.delta":
            print(event.data.content, flush=True, end="")

La bajada de gradiente es como bajar una montaña poco a poco, siempre eligiendo el camino más empinado hacia abajo. En el aprendizaje automático, usamos esta técnica para ajustar los parámetros del modelo con el fin de minimizar el error, es decir, encontrar el mejor "valle" que representa la solución óptima.

## <font color="gree">``4.`` Usando a pesquisa na web para obter informações em tempo real</font>

<img src="https://docs.mistral.ai/img/websearch_connector.png" width="400" alt="Diagrama de conexão para pesquisa web">

* A maioria dos LLMs vive em 2024.
* Têm alucinações quando questionados sobre os eventos de 2025.

In [28]:
research_agent = client.beta.agents.create(
    model="mistral-medium-latest",
    name="ml-research-agent",
    description="Especialista em pesquisa de Machine Learning (ML)",
    tools=[{"type": "web_search"}],
)

search_response = client.beta.conversations.start(
    agent_id=research_agent.id, inputs="Resumo das últimas melhorias em Transformer em 2025"
)

In [29]:
print(search_response.outputs)

[ToolExecutionEntry(name='web_search', arguments='{"query": "Resumo das últimas melhorias em Transformer em 2025"}', object='entry', type='tool.execution', created_at=datetime.datetime(2025, 7, 17, 19, 8, 23, 371202, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2025, 7, 17, 19, 8, 24, 723588, tzinfo=TzInfo(UTC)), id='tool_exec_019819c9a9cb71528b028a6cab5c42c0', info={}), MessageOutputEntry(content=[TextChunk(text='Em 2025, as melhorias nos modelos Transformer continuaram a avançar significativamente, impulsionando ainda mais a capacidade e a eficiência desses modelos em diversas aplicações de inteligência artificial. Aqui estão algumas das principais melhorias e tendências observadas:  \n  \n1. **Eficiência Computacional**:  \n   - Os modelos Transformer estão se tornando mais eficientes em termos computacionais. Novas técnicas de compressão de modelos e otimização de hardware permitem que esses modelos sejam executados em dispositivos com menos recursos, como smartphones e disp

In [30]:
search_response.outputs[1].content

[TextChunk(text='Em 2025, as melhorias nos modelos Transformer continuaram a avançar significativamente, impulsionando ainda mais a capacidade e a eficiência desses modelos em diversas aplicações de inteligência artificial. Aqui estão algumas das principais melhorias e tendências observadas:  \n  \n1. **Eficiência Computacional**:  \n   - Os modelos Transformer estão se tornando mais eficientes em termos computacionais. Novas técnicas de compressão de modelos e otimização de hardware permitem que esses modelos sejam executados em dispositivos com menos recursos, como smartphones e dispositivos IoT, sem sacrificar significativamente o desempenho', type='text'),
 ToolReferenceChunk(tool='web_search', title='Arquitetura transformers: O que é, Como Funciona e Benefícios | MOD', type='tool_reference', url='https://mercadoonlinedigital.com/blog/transformers/', favicon='https://imgs.search.brave.com/dQYtVZHFYeV-X5QHMrfmO35kuZEZuCm0xq0eyIkS8pw/rs:fit:32:32:1:0/g:ce/aHR0cDovL2Zhdmlj/b25zLnNlYXJ

* ``web_search`` retorna texto com citações.
* Isso requer formatação especial.

In [31]:
markdown_text = ""

for result in search_response.outputs[1].content:
    if hasattr(result, "text"):
        markdown_text += result.text
    if hasattr(result, "tool"):
        markdown_text += f" ([{result.title}]({result.url}))"  # Links are given []()

In [32]:
from IPython.display import Markdown

Markdown(markdown_text)

Em 2025, as melhorias nos modelos Transformer continuaram a avançar significativamente, impulsionando ainda mais a capacidade e a eficiência desses modelos em diversas aplicações de inteligência artificial. Aqui estão algumas das principais melhorias e tendências observadas:  
  
1. **Eficiência Computacional**:  
   - Os modelos Transformer estão se tornando mais eficientes em termos computacionais. Novas técnicas de compressão de modelos e otimização de hardware permitem que esses modelos sejam executados em dispositivos com menos recursos, como smartphones e dispositivos IoT, sem sacrificar significativamente o desempenho ([Arquitetura transformers: O que é, Como Funciona e Benefícios | MOD](https://mercadoonlinedigital.com/blog/transformers/)).  
  
2. **Avanços em Multimodalidade**:  
   - A integração de dados multimodais (texto, imagem, áudio) em um único modelo Transformer tem visto avanços significativos. Modelos como o CLIP (Contrastive Language–Image Pretraining) e DALL-E estão sendo aprimorados para entender e gerar conteúdo multimodal de maneira mais coerente e contextualizada. Isso permite aplicações mais robustas em áreas como geração de arte, análise de mídia social e assistentes virtuais mais inteligentes ([What are Transformers? - Transformers in Artificial Intelligence Explained - AWS](https://aws.amazon.com/what-is/transformers-in-artificial-intelligence/)).  
  
3. **Melhorias em Atenção e Memória**:  
   - Novas variantes da camada de atenção, como a atenção esparsa e a atenção baseada em memória, estão sendo desenvolvidas para melhorar a eficiência e a capacidade dos modelos Transformer. Essas inovações permitem que os modelos lidem melhor com sequências mais longas e complexas, melhorando a precisão em tarefas como tradução automática e sumarização de textos longos ([How Transformers Work: A Detailed Exploration of Transformer Architecture | DataCamp](https://www.datacamp.com/tutorial/how-transformers-work)).  
  
4. **Aplicações em Saúde**:  
   - Os modelos Transformer estão sendo cada vez mais aplicados na área da saúde, desde a análise de prontuários médicos até a descoberta de novos medicamentos. A capacidade de processar grandes volumes de dados e identificar padrões complexos está ajudando a acelerar pesquisas e melhorar diagnósticos. Em 2025, espera-se que mais hospitais e instituições de pesquisa adotem essas tecnologias para melhorar a precisão e a velocidade dos tratamentos ([O que é um Modelo Transformer? | Blog da NVIDIA](https://blog.nvidia.com.br/blog/o-que-e-um-modelo-transformer/)).  
  
5. **Personalização e Adaptabilidade**:  
   - Os modelos Transformer estão se tornando mais adaptáveis a contextos específicos e personalizáveis para usuários individuais. Isso é particularmente útil em aplicações como assistentes pessoais, onde o modelo pode aprender e adaptar-se às preferências e necessidades específicas de um usuário ao longo do tempo.  
  
6. **Avanços em Processamento de Linguagem Natural (PLN)**:  
   - A compreensão contextual e a geração de linguagem natural continuam a melhorar. Modelos como o GPT-5 estão sendo treinados com conjuntos de dados ainda maiores e mais diversos, permitindo uma compreensão mais profunda e uma geração de texto mais coerente e contextualmente apropriada. Isso está levando a avanços em áreas como chatbots, tradução automática e análise de sentimentos ([O que é um GPT? Introdução aos Transformers na Inteligência Artificial](https://www.guilhermefavaron.com.br/post/transformers-inteligencia-artificial)).  
  
7. **Integração com Outras Tecnologias**:  
   - Os modelos Transformer estão sendo integrados com outras tecnologias emergentes, como blockchain e computação quântica, para criar sistemas ainda mais poderosos e seguros. Essa integração está abrindo novas possibilidades em áreas como segurança cibernética e análise de dados em tempo real.  
  
Essas melhorias estão posicionando os modelos Transformer como uma das tecnologias mais influentes e transformadoras na área de inteligência artificial, com aplicações que se estendem por quase todos os setores da economia e da sociedade.

In [33]:
# Continuar a conversa após os resultados:
follow_up = client.beta.conversations.append(
    conversation_id=search_response.conversation_id,
    inputs="Resuma os resultados em 2 frases."
)

In [34]:
print(follow_up.outputs[0].content)

Em 2025, os modelos Transformer continuaram a evoluir com melhorias significativas em eficiência computacional, capacidades multimodais e aplicações em saúde, tornando-se mais adaptáveis e integrados com outras tecnologias emergentes. Esses avanços estão expandindo suas aplicações em diversas áreas, desde assistentes pessoais até descoberta de medicamentos, consolidando sua posição como uma das tecnologias mais influentes em inteligência artificial.


## <font color="gree">``5.`` Usando execução de código para tarefas complexas</font>

<img src="https://docs.mistral.ai/img/code_interpreter_connector.png" width="400" alt="Diagrama de conexão do Code Interpreter">

In [35]:
code_agent = client.beta.agents.create(
    model="mistral-medium-2505",
    name="coding-assistant",
    description="Especialista em execução de código",
    tools=[{"type": "code_interpreter"}],
)

message = """Resolva e visualize a equação do conjunto de Mandelbrot z = z² + c. 
            Calcule a convergência para uma grade de plano complexo, crie uma 
            bela arte de parede fractal com mapeamento de cores personalizado."""

code_response = client.beta.conversations.start(
    agent_id=code_agent.id,
    inputs=message,
)

In [36]:
code_response.outputs

[MessageOutputEntry(content='Para resolver e visualizar o conjunto de Mandelbrot, precisamos iterar sobre uma grade de números complexos e determinar se a sequência \\( z = z^2 + c \\) converge ou diverge. Podemos então mapear os resultados para criar uma visualização colorida do fractal.\n\nVamos prosseguir com o código para gerar essa visualização.', object='entry', type='message.output', created_at=datetime.datetime(2025, 7, 17, 19, 23, 19, 269715, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2025, 7, 17, 19, 23, 20, 682289, tzinfo=TzInfo(UTC)), id='msg_019819d75565703e8d4493e1fdc47bee', agent_id='ag_019819d75039708580e582f29546ad0d', model='mistral-medium-2505', role='assistant'),
 ToolExecutionEntry(name='code_interpreter', arguments='{"code": "import numpy as np\\nimport matplotlib.pyplot as plt\\n\\n# Definindo a função do conjunto de Mandelbrot\\ndef mandelbrot(c, max_iter):\\n    z = c\\n    for n in range(max_iter):\\n        if abs(z) > 2:\\n            return n\\n   

In [37]:
print(code_response.outputs[1])

name='code_interpreter' arguments='{"code": "import numpy as np\\nimport matplotlib.pyplot as plt\\n\\n# Definindo a função do conjunto de Mandelbrot\\ndef mandelbrot(c, max_iter):\\n    z = c\\n    for n in range(max_iter):\\n        if abs(z) > 2:\\n            return n\\n        z = z*z + c\\n    return max_iter\\n\\n# Configurações da grade e iterações\\nxmin, xmax = -2.0, 1.0\\nymin, ymax = -1.5, 1.5\\nwidth, height = 800, 800\\nmax_iter = 256\\n\\n# Criando a grade de números complexos\\nx = np.linspace(xmin, xmax, width)\\ny = np.linspace(ymin, ymax, height)\\nX, Y = np.meshgrid(x, y)\\nC = X + 1j * Y\\n\\n# Calculando o conjunto de Mandelbrot\\nmandelbrot_set = np.array([[mandelbrot(c, max_iter) for c in row] for row in C])\\n\\n# Visualização com mapeamento de cores personalizado\\nplt.figure(figsize=(10, 10))\\nplt.imshow(mandelbrot_set, extent=(xmin, xmax, ymin, ymax), cmap=\'twilight_shifted\', interpolation=\'bilinear\')\\nplt.colorbar(label=\'Iteração em que |z| > 2\')\\n

In [38]:
code_snippet = code_response.outputs[1].info["code"]

print(code_snippet)

import numpy as np
import matplotlib.pyplot as plt

# Definindo a função do conjunto de Mandelbrot
def mandelbrot(c, max_iter):
    z = c
    for n in range(max_iter):
        if abs(z) > 2:
            return n
        z = z*z + c
    return max_iter

# Configurações da grade e iterações
xmin, xmax = -2.0, 1.0
ymin, ymax = -1.5, 1.5
width, height = 800, 800
max_iter = 256

# Criando a grade de números complexos
x = np.linspace(xmin, xmax, width)
y = np.linspace(ymin, ymax, height)
X, Y = np.meshgrid(x, y)
C = X + 1j * Y

# Calculando o conjunto de Mandelbrot
mandelbrot_set = np.array([[mandelbrot(c, max_iter) for c in row] for row in C])

# Visualização com mapeamento de cores personalizado
plt.figure(figsize=(10, 10))
plt.imshow(mandelbrot_set, extent=(xmin, xmax, ymin, ymax), cmap='twilight_shifted', interpolation='bilinear')
plt.colorbar(label='Iteração em que |z| > 2')
plt.title('Conjunto de Mandelbrot')
plt.xlabel('Re(c)')
plt.ylabel('Im(c)')
plt.show()


In [39]:
file_chunk = code_response.outputs[2].content[0]

file_chunk

ToolFileChunk(tool='code_interpreter', file_id='d7780444-8745-43e9-9723-eaa6bd730607', type='tool_file', file_name='plot_0.png', file_type='png')

In [40]:
file_id = file_chunk.file_id

In [41]:
file_bytes = client.files.download(file_id=file_id).read()

with open("mandelbrot.png", "wb") as f:
    f.write(file_bytes)

# Conclusão e próximos passos

* Confira a [documentação](https://mistral.ai/news/agents-api)

* Explore outras ferramentas como [document search](https://docs.mistral.ai/agents/connectors/document_library/) e [image generation](https://docs.mistral.ai/agents/connectors/image_generation/)

* Aprenda como usar [function calling](https://docs.mistral.ai/agents/function_calling/)

* Construa [sistemas de multi-agentes](https://docs.mistral.ai/agents/handoffs/)